# 18.4: House Model Interpretation (Question 3)

In [6]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

In [7]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
house_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

## Data Exploration And Cleaning

In [3]:
house_df.head(10)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,...,0,None,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,None,IR1,Lvl,AllPub,...,0,None,None,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,1,2008,WD,Normal,118000


In [8]:
non_numeric_columns = house_df.select_dtypes(['object']).columns
print(non_numeric_columns)
print("The number of non-numerical columns is {}".format(len(non_numeric_columns)))

Index(['mszoning', 'street', 'alley', 'lotshape', 'landcontour', 'utilities',
       'lotconfig', 'landslope', 'neighborhood', 'condition1', 'condition2',
       'bldgtype', 'housestyle', 'roofstyle', 'roofmatl', 'exterior1st',
       'exterior2nd', 'masvnrtype', 'exterqual', 'extercond', 'foundation',
       'bsmtqual', 'bsmtcond', 'bsmtexposure', 'bsmtfintype1', 'bsmtfintype2',
       'heating', 'heatingqc', 'centralair', 'electrical', 'kitchenqual',
       'functional', 'fireplacequ', 'garagetype', 'garagefinish', 'garagequal',
       'garagecond', 'paveddrive', 'poolqc', 'fence', 'miscfeature',
       'saletype', 'salecondition'],
      dtype='object')
The number of non-numerical columns is 43


In [9]:
house_df_dummified = pd.get_dummies(house_df, drop_first=True, columns=['salecondition'], prefix='salecondition')

In [10]:
house_df_dummified = pd.get_dummies(house_df_dummified, drop_first=True, columns=['mszoning'], prefix='mszoning')

In [11]:
house_df_dummified.columns

Index(['id', 'mssubclass', 'lotfrontage', 'lotarea', 'street', 'alley',
       'lotshape', 'landcontour', 'utilities', 'lotconfig', 'landslope',
       'neighborhood', 'condition1', 'condition2', 'bldgtype', 'housestyle',
       'overallqual', 'overallcond', 'yearbuilt', 'yearremodadd', 'roofstyle',
       'roofmatl', 'exterior1st', 'exterior2nd', 'masvnrtype', 'masvnrarea',
       'exterqual', 'extercond', 'foundation', 'bsmtqual', 'bsmtcond',
       'bsmtexposure', 'bsmtfintype1', 'bsmtfinsf1', 'bsmtfintype2',
       'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 'heating', 'heatingqc',
       'centralair', 'electrical', 'firstflrsf', 'secondflrsf', 'lowqualfinsf',
       'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath', 'halfbath',
       'bedroomabvgr', 'kitchenabvgr', 'kitchenqual', 'totrmsabvgrd',
       'functional', 'fireplaces', 'fireplacequ', 'garagetype', 'garageyrblt',
       'garagefinish', 'garagecars', 'garagearea', 'garagequal', 'garagecond',
       'paveddrive', 'woo

In [12]:
df_with_dummies = pd.get_dummies(house_df, prefix_sep='salecondition', columns=['salecondition'])

In [15]:
columns_to_fit = [x for x in house_df_dummified.columns if 'mszoning' in x or 'salecondition' in x]

In [18]:
house_df_dummified[columns_to_fit].columns

Index(['salecondition_AdjLand', 'salecondition_Alloca', 'salecondition_Family',
       'salecondition_Normal', 'salecondition_Partial', 'mszoning_FV',
       'mszoning_RH', 'mszoning_RL', 'mszoning_RM'],
      dtype='object')

## OLS Model

## Attempt 1 experiment: Y is the target variable and using the original data set.

In [25]:
Y = house_df['saleprice']
# X is the feature set
X = house_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf']]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     926.5
Date:                Tue, 28 Apr 2020   Prob (F-statistic):               0.00
Time:                        13:09:44   Log-Likelihood:                -17499.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1454   BIC:                         3.504e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -9.907e+04   4638.450    -21.359      0.000   -1.08e+05      -9e+04
overallqual  2.364e+04   1072.532     22.037      0.000    2.15e+04    2.57e+04
grlivarea      45.3458      2.489     18.218      0.000      40.463      50.228
garagecars   1.454e+04   3022.681      4.812      0.000    8615.034    2.05e+04
garagearea     17.1334     10.468      1.637      0.102      -3.401      37.668
totalbsmtsf    31.5015      2.904     10.848      0.000      25.805      37.198
==============================================================================
Omnibus:                      438.146   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            40648.199
Skew:                          -0.338   Prob(JB):                         0.00
Kurtosis:                      28.841   Cond. No.                     9.23e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.23e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Attempt 2: Experiment using new dataset house_df_dummified

In [31]:
# Y is the target variable
Y = house_df_dummified['saleprice']
# X is the feature set
X = house_df_dummified[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf'] + columns_to_fit]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.777
Model:                            OLS   Adj. R-squared:                  0.775
Method:                 Least Squares   F-statistic:                     359.4
Date:                Tue, 28 Apr 2020   Prob (F-statistic):               0.00
Time:                        13:24:36   Log-Likelihood:                -17449.
No. Observations:                1460   AIC:                         3.493e+04
Df Residuals:                    1445   BIC:                         3.501e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.155e+05   1.25e+04     -9.213      0.000    -1.4e+05   -9.09e+04
overallqual            2.218e+04   1084.790     20.448      0.000    2.01e+04    2.43e+04
grlivarea                46.8749      2.439     19.220      0.000      42.091      51.659
garagecars              1.32e+04   2957.642      4.461      0.000    7393.420     1.9e+04
garagearea               11.7513     10.258      1.146      0.252      -8.371      31.874
totalbsmtsf              26.8902      2.913      9.230      0.000      21.175      32.605
salecondition_AdjLand  1.121e+04   1.94e+04      0.579      0.563   -2.68e+04    4.92e+04
salecondition_Alloca   8648.3893   1.16e+04      0.744      0.457   -1.41e+04    3.14e+04
salecondition_Family  -1.074e+04   9278.461     -1.157      0.247   -2.89e+04    7461.089
salecondition_Normal   9782.6002   3986.679      2.454      0.014    1962.303    1.76e+04
salecondition_Partial  3.341e+04   5386.012      6.203      0.000    2.28e+04     4.4e+04
mszoning_FV            1.729e+04   1.32e+04      1.305      0.192   -8697.574    4.33e+04
mszoning_RH             1.22e+04   1.53e+04      0.795      0.427   -1.79e+04    4.23e+04
mszoning_RL            2.552e+04   1.23e+04      2.077      0.038    1415.315    4.96e+04
mszoning_RM            6295.8410   1.25e+04      0.505      0.614   -1.82e+04    3.08e+04
==============================================================================
Omnibus:                      470.227   Durbin-Watson:                   1.970
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            55231.633
Skew:                          -0.395   Prob(JB):                         0.00
Kurtosis:                      33.121   Cond. No.                     5.57e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.57e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""


The statistically significant variables are 'overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'salecondition_Normal','salecondition_Partial', 'mszoning_FV' and 'mszoning_RL'. All the other variables are statistically insignificant. Consequently, their coefficients are statistically zero. According to the estimation results:

A 1 point increase in overallqual results in \$22180 increase in sale price.
A 1 point increase in grlivarea results in \$46.87 increase in sale price.
A 1 point increase in garagecars results in \$13200 increase in sale price.
A 1 point increase in totalbsmtsf results in \$26.89 increase in sale price.
The average sale price is higher in mszoning_FV and mszoning_RL than the base zone by \$12200 and \$25520 respectively. In conclusion, the highest average house price is in mszoning_RL.

### Attempt 3: 

In [16]:
# Y is the target variable
Y = house_df['saleprice']
# X is the feature set which includes
# dummified variables
X = house_df_dummified[columns_to_fit]

# We create a LinearRegression model object
# from scikit-learn's linear_model module.
lrm = linear_model.LinearRegression()

# fit method estimates the coefficients using OLS
lrm.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', lrm.coef_)
print('\nIntercept: \n', lrm.intercept_)


Coefficients: 
 [-59476.95179673  21815.66507042   3488.48680105  21559.90029216
 110544.70413423  99940.51735703  53013.47017305  99879.47842064
  41578.01642803]

Intercept: 
 63722.4733760946


The estimated model is:
saleprice = 63722.47 -59476.95salecondition_AdjLand + 21815.67salecondition_Alloca + 3488.49salecondition_Family + 21559.9salecondition_Normal +110544.70salecondition_Partial + 99940.52mszoning_FV + 53013.47mszoning_RH +99879.49mszoning_RL + 41578.02mszoning_RM

### Attempt 4: 
We observed that the statistically significant variables are 'overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'salecondition_Normal','salecondition_Partial', 'mszoning_FV' and 'mszoning_RL'.Now let's exclude statistically insignificant variables from the second model and re-estimate it:

In [34]:
# Y is the target variable
Y = house_df_dummified['saleprice']
# X is the feature set
X = house_df_dummified[['overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'salecondition_Normal',
                        'salecondition_Partial','mszoning_FV', 'mszoning_RL']]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.775
Method:                 Least Squares   F-statistic:                     629.1
Date:                Tue, 28 Apr 2020   Prob (F-statistic):               0.00
Time:                        13:38:08   Log-Likelihood:                -17451.
No. Observations:                1460   AIC:                         3.492e+04
Df Residuals:                    1451   BIC:                         3.497e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.099e+05   5481.975    -20.043      0.000   -1.21e+05   -9.91e+04
overallqual            2.213e+04   1073.666     20.616      0.000       2e+04    2.42e+04
grlivarea                47.3288      2.420     19.558      0.000      42.582      52.076
garagecars             1.581e+04   1715.595      9.214      0.000    1.24e+04    1.92e+04
totalbsmtsf              27.2484      2.839      9.598      0.000      21.680      32.817
salecondition_Normal   1.049e+04   3428.806      3.060      0.002    3765.948    1.72e+04
salecondition_Partial  3.447e+04   4990.899      6.906      0.000    2.47e+04    4.43e+04
mszoning_FV            1.148e+04   5528.062      2.076      0.038     631.779    2.23e+04
mszoning_RL            1.934e+04   2791.382      6.930      0.000    1.39e+04    2.48e+04
==============================================================================
Omnibus:                      456.549   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            51234.082
Skew:                          -0.345   Prob(JB):                         0.00
Kurtosis:                      32.013   Cond. No.                     1.24e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.24e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Assessing Statistical Significance of Coefficients

### We will run the stats model to fit an OLS model. From the results, we will be able to interpret the dummy variable, continuous variable, and the interactions between explanatory variables. 

In [35]:
#Funtion .summary() returns t-statistic and p-value
import statsmodels.api as sm

# We fit an OLS model using statsmodels
results = sm.OLS(Y, X).fit()

# We print the summary results.
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.775
Method:                 Least Squares   F-statistic:                     629.1
Date:                Tue, 28 Apr 2020   Prob (F-statistic):               0.00
Time:                        13:38:34   Log-Likelihood:                -17451.
No. Observations:                1460   AIC:                         3.492e+04
Df Residuals:                    1451   BIC:                         3.497e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.099e+

## NEW OLS MODEL: Results 
saleprice = -109,900 + 22,130overallqual + 47.3288grlivarea + 1,581garagecars + 27.2totalbsmtsf +  1,049salecondition_Normal + 3,447salecondition_Partial + 1,148mszoning_FV + 1,934mszoning_RL

The results resemble the previous model's results. The estimated coefficients are close to the previous model except the dummy variables. For example, 'grlivarear' only decreased by less than 1 point (46.8 to 47.32). 

The coefficients of the dummies changed relatively larger than the other variables. For example, mszoning_RL changed from 2,552 to 1,934. And mszoning_FV changed from 1,729 to 1,148.